<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
<center>
Автор материала: Екатерина Демидова, Data Scientist @ Segmento <br>


Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.
Автор материала: 

# <center>Домашнее задание № 2 (демо). <br> Визуальный анализ данных о публикациях на Хабрахабре</center>

**Заполните код в клетках и ответьте на вопросы в [веб-форме](https://docs.google.com/forms/d/1y8qj5iYpaVhczjWnZuHd5NRR_sdCGy3eTF5n-qPKknQ).**

In [1]:
import pandas as pd
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

## Загрузка и знакомство с данными

Для работы вам понадобятся предобработанные данные нашего учебного конкурса на kaggle [«Прогноз популярности статьи на Хабре»](https://www.kaggle.com/c/howpop-habrahabr-favs).

In [ ]:
df = pd.read_csv('data.csv')

In [3]:
df.shape

(134137, 17)

In [2]:
df.head(3).T

NameError: name 'df' is not defined

Избавимся сразу от переменных, названия которых заканчиваются на `_lognorm` (нужны для соревнования на Kaggle). Выберем их с помощью `filter()` и удалим `drop`-ом:

In [5]:
df.drop(filter(lambda c: c.endswith('_lognorm'), df.columns), axis = 1, inplace = True)

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
post_id,134137.0,181307.054265,81766.350702,18284.0,115565.0,182666.0,257401.0,314088.0
content_len,134137.0,7516.929699,8724.771640,1.0,2179.0,4949.0,9858.0,352471.0
comments,134137.0,39.625994,62.398958,0.0,7.0,19.0,48.0,2266.0
favs,134137.0,71.999866,145.854135,0.0,8.0,27.0,78.0,8718.0
views,134137.0,16631.013084,31479.819691,3.0,2100.0,7600.0,18700.0,1730000.0
votes_plus,133566.0,35.536888,42.461073,0.0,11.0,22.0,45.0,1466.0
votes_minus,133566.0,8.050035,11.398282,0.0,2.0,5.0,10.0,445.0


In [7]:
df.describe(include = ['object', 'bool']).T

,count,unique,top,freq
url,134137,134137,https://habrahabr.ru/company/Ciklum/blog/247483/,1
domain,134137,2,habrahabr.ru,97048
published,134137,130291,2011-06-14 15:52:00,39
author,97657,22077,@alizar,5292
flow,97048,6,develop,53318
polling,134137,2,False,129412
title,134137,133628,"Интересные события, произошедшие в выходные",56


In [8]:
sns.set_style("dark")
sns.set_palette("RdBu")
sns.set_context("notebook", font_scale = 1.5, rc = { "figure.figsize" : (15, 5), "axes.titlesize" : 18 })

NameError: name 'sns' is not defined

Столбец **`published`** (время публикации) содержит строки. Чтобы мы могли работать с этими данными как с датой/временем публикации, приведём их к типу `datetime`:

In [9]:
print(df.published.dtype)
df['published'] = pd.to_datetime(df.published, yearfirst = True)
print(df.published.dtype)

object
datetime64[ns]


Создадим несколько столбцов на основе данных о времени публикации:

In [10]:
df['year'] = [d.year for d in df.published]
df['month'] = [d.month for d in df.published]

df['dayofweek'] = [d.isoweekday() for d in df.published]
df['hour'] = [d.hour for d in df.published]

-----
Теперь Ваша очередь. В каждом пункте предлагается построить картинку и с ее помощью ответить на вопрос в [форме](https://docs.google.com/forms/d/e/1FAIpQLSf3b5OG8zX_nLQBQ-t20c6M5Auz-VUL-yxj8Fm9_o_XWDBTrg/viewform?c=0&w=1). Конечно, можно попытаться ответить на все вопросы только с Pandas, без картинок, но мы советуем Вам потренироваться строить (красивые) визуализации.   

## 1\. В каком месяце (и какого года) было больше всего публикаций?

* март 2016
* март 2015
* апрель 2015
* апрель 2016

In [11]:
df['mon_yr'] = [str(p)[:7] for p in df.published]
print(df.mon_yr.value_counts())
mon = df.mon_yr.value_counts().index[0]
print(mon)

NameError: name 'plt' is not defined

## 2\. Проанализируйте публикации в месяце из предыдущего вопроса

Выберите один или несколько вариантов:

* Один или несколько дней сильно выделяются из общей картины
* На хабре _всегда_ больше статей, чем на гиктаймсе
* По субботам на гиктаймс и на хабрахабр публикуют примерно одинаковое число статей

Подсказки: постройте график зависимости числа публикаций от дня; используйте параметр `hue`; не заморачивайтесь сильно с ответами и не ищите скрытого смысла :)

In [ ]:
df_mon = df[df.mon_yr == mon]
df_mon['day'] = [p.day for p in df_mon.published]
fig = plt.figure(figsize=(20,20))
fig.add_subplot(1,1,1)
sns.countplot(x = 'day', data = df_mon);

In [ ]:
fig = plt.figure(figsize=(15,15))
fig.add_subplot(1,1,1)
sns.countplot(x = 'day', hue = 'domain', data = df_mon);

In [ ]:
fig = plt.figure(figsize=(15,15))
fig.add_subplot(1,1,1)
sns.countplot(x = 'dayofweek', hue = 'domain', data = df_mon);

## 3\. Когда лучше всего публиковать статью?

* Больше всего просмотров набирают статьи, опубликованные в 12 часов дня
* У опубликованных в 10 утра постов больше всего комментариев
* Больше всего просмотров набирают статьи, опубликованные в 6 часов утра
* Максимальное число комментариев на гиктаймсе набрала статья, опубликованная в 9 часов вечера
* На хабре дневные статьи комментируют чаще, чем вечерние

In [12]:
print(df.groupby('hour')['views'].mean().sort_values(ascending=False)[:5])

NameError: name 'plt' is not defined

In [ ]:
print(df.groupby('hour')['comments'].mean().sort_values(ascending=False)[:5])

In [ ]:
print(df[df.domain == 'habrahabr.ru'].groupby('hour')['comments'].mean().sort_values(ascending=False)[:5])

In [ ]:
df[df.domain == 'habrahabr.ru'].groupby('hour')[['comments']].mean().plot();

## 4\. Кого из топ-20 авторов чаще всего минусуют?

* <b>@Mordatyj</b>
* @Mithgol
* @alizar
* @ilya42

In [ ]:
df[df.author.isin(['@Mordatyj','@Mithgol', '@alizar', '@ilya42'])].groupby('author')[['votes_minus']].mean().sort_values('votes_minus', ascending = False)

## 5\. Сравните субботы и понедельники

Правда ли, что по субботам авторы пишут в основном днём, а по понедельникам — в основном вечером? <b>Нет</b>

In [ ]:
fig = plt.figure(figsize=(20,20))
fig.add_subplot(1,1,1)
pl = sns.countplot(y = 'hour', hue = 'dayofweek', data = df[df.dayofweek.isin([1, 6])], palette="Set2");
pl.set_title("Количество публикаций за час", fontweight = 'bold');